In [1]:
import os
from fastai.text import * 

In [7]:
torch.cuda.device_count()

1

In [2]:
LANG = 'en'

In [3]:
proj_path = Path(os.getcwd()).parents[1]

In [4]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [5]:
df = pd.read_csv(data_path/'all_bot_human_data.csv')

In [6]:
df.head()

,bot_human,text
0,human,@ChadPergram Can he call his mom?
1,human,@birdonawireart ended up choosing a bear eatin...
2,human,RT @IntJudoFed: @HeymanHustle @WWE @ufc @Brock...
3,bot,Read About Finance Trade Secrets http://t.co/...
4,bot,"21:4 An high look, and a just bath."


In [8]:
data_lm = TextLMDataBunch.from_csv(data_path, 'all_bot_human_data.csv')

In [9]:
data_clas = TextClasDataBunch.from_csv(data_path, 'all_bot_human_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [10]:
data_lm.save(f'data_lm_export_{LANG}.pkl')
data_clas.save(f'data_clas_export_{LANG}.pkl')

In [11]:
data_lm = load_data(data_path, fname=f'data_lm_export_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{LANG}.pkl', bs=16)

In [12]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,3.515250,3.248378,0.462042


In [13]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,3.054218,2.920586,0.509641


In [14]:
learn.predict("This is a review about", n_words=10)

'This is a review about the History of David Gates http :'

In [15]:
learn.save_encoder('ft_enc')

In [16]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder('ft_enc')

In [17]:
data_clas.show_batch()

text,target
xxbos @pizster @bendleshnitz xxup proud ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤,human
xxbos ( xxmaj entry - xxmaj level / xxmaj new xxmaj graduate ) xxmaj software xxmaj engineer – xxup .net / c # / xxmaj java : ( xxmaj entry - xxmaj level / xxmaj new xxmaj graduate ) xxmaj software xxmaj engineer – xxup .net / c # / xxmaj java at xxup petsmart xxmaj skip xxmaj navigation – xxmaj saved xxmaj jobs – xxrep 12 * – –,bot
"xxbos xxmaj sr xxup qa xxmaj consultant ( xxmaj for c # xxmaj application / xxmaj miami , xxup fl : xxmaj job xxmaj tittle : xxup qa for c # xxmaj applications xxmaj number of xxmaj contractors : 2 xxmaj rate : $ xxmaj market / xxmaj hr xxmaj position : xxmaj contract xxmaj duration : xxmaj long - term xxmaj location : xxmaj miami , xxup fl xxmaj",bot
"xxbos xxmaj developer : xxmaj development xxmaj expert for xxup sap xxmaj labs , xxup llc ( xxmaj palo xxmaj alto , xxup ca ) resp for dvlpmnt focus on dsgn , codng , tstng , & & qlty assur of cmplx prod features in dvlpmnt team . xxmaj bach in xxmaj comp xxmaj sci , xxmaj eng , xxmaj math , or rltd field & & 7 yrs exp",bot
"xxbos xxmaj simulation xxmaj analyst , xxmaj special xxmaj operations xxmaj forces xxmaj job : xxmaj date : xxmaj mar 31 , 2018 xxmaj location : xxmaj quantico , xxup va , xxup us xxmaj company : xxup saic xxmaj simulation xxmaj analyst , xxmaj special xxmaj operations xxmaj forces ( xxmaj job xxmaj xxunk ) xxmaj description : xxup saic is seeking a xxmaj simulation xxmaj analyst to support",bot


In [19]:
learn_class.fit_one_cycle(5, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.399654,0.282305,0.883472


Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/bot_gender_gpu/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/miniconda3/envs/bot_gender_gpu/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/miniconda3/envs/bot_gender_gpu/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/miniconda3/envs/bot_gender_gpu/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/bot_gender_gpu/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/miniconda3/envs/bot_gender_gpu/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:of

KeyboardInterrupt: 